In [ ]:
import io
import pandas as pd

: 

In [ ]:
csv_data='''
"name","working_hours","salary","leaves"
"tony","5.0","60","3"
"mary","7","100","5"
'''

: 

In [3]:
data=pd.read_csv(io.StringIO(csv_data))
data

,name,working_hours,salary,leaves
0,tony,5.0,60,3
1,mary,7.0,100,5


In [4]:
blank_data='''
name,price,circular
advanced,150,
reader's digest,,150million
'''

simulated_csv=pd.read_csv(io.StringIO(blank_data))
simulated_csv

,name,price,circular
0,advanced,150.0,NaN
1,reader's digest,NaN,150million


In [5]:
simulated_csv.dropna(axis="columns")

,name
0,advanced
1,reader's digest


In [6]:
simulated_csv.dropna(subset=["price"])

,name,price,circular
0,advanced,150.0,NaN


In [7]:
pd.DataFrame(pd.read_csv(io.StringIO(blank_data)))["price"]

0    150.0
1      NaN
Name: price, dtype: float64

In [8]:
simulated_csv.fillna(pd.DataFrame(pd.read_csv(io.StringIO(blank_data)))["price"].mean())

,name,price,circular
0,advanced,150.0,150.0
1,reader's digest,150.0,150million


## catagroical data

In [69]:
columns=["device type","price","resolution(definition)"]
devices=pd.DataFrame([["notebook","$1000","4K"],["desktop","$2999","1080p"],["phone","$500","720p"]],columns=columns)
devices

,device type,price,resolution(definition)
0,notebook,$1000,4K
1,desktop,$2999,1080p
2,phone,$500,720p


#### 建一個對照表，轉換數值

In [72]:
data_transform={"4K":"3840*2160","1080p":"1920*1080","720p":"720*480"}
data_transform

{'4K': '3840*2160', '1080p': '1920*1080', '720p': '720*480'}

##### seris map

In [73]:
devices["resolution(definition)"].map(data_transform)

0    3840*2160
1    1920*1080
2      720*480
Name: resolution(definition), dtype: object

#### dataframe.map

In [75]:
devices.map(lambda x: len(x))

,device type,price,resolution(definition)
0,8,5,2
1,7,5,5
2,5,4,4


### one hot encoding

In [88]:
one_hot_encoding=pd.get_dummies(devices["resolution(definition)"],prefix="resolution")
devices

,device type,price,resolution(definition)
0,notebook,$1000,4K
1,desktop,$2999,1080p
2,phone,$500,720p


In [92]:
dropped_devices=devices.drop(["resolution(definition)"],axis=1)
pd.concat([dropped_devices, one_hot_encoding], axis=1)

,device type,price,resolution_1080p,resolution_4K,resolution_720p
0,notebook,$1000,False,True,False
1,desktop,$2999,True,False,False
2,phone,$500,False,False,True


#### feature scaling 
$$
x_{(norm)}=\frac{x_i-x_\mu}{x_{max}-x_{min}}
$$

In [94]:
from sklearn import datasets

In [103]:
iris=datasets.load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [154]:
x=pd.DataFrame(data=iris["data"],columns=iris['feature_names'])

In [155]:
x

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


#### 對sepal length做normalization

In [171]:
#initialize the datasets

x=pd.DataFrame(data=iris["data"],columns=iris["feature_names"])

In [165]:
def normalization(a,data_name):
    return (a-data_name["sepal length (cm)"].min())/(data_name["sepal length (cm)"].max()-data_name["sepal length (cm)"].min())
x["sepal length (cm)"].apply(lambda i: normalization(i,x))

0      0.222222
1      0.166667
2      0.111111
3      0.083333
4      0.194444
         ...   
145    0.666667
146    0.555556
147    0.611111
148    0.527778
149    0.444444
Name: sepal length (cm), Length: 150, dtype: float64

or

In [164]:
x["sepal length (cm)"] = (x["sepal length (cm)"]-x["sepal length (cm)"].min())/(x["sepal length (cm)"].max()-x["sepal length (cm)"].min())
x

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.222222,3.5,1.4,0.2
1,0.166667,3.0,1.4,0.2
2,0.111111,3.2,1.3,0.2
3,0.083333,3.1,1.5,0.2
4,0.194444,3.6,1.4,0.2
...,...,...,...,...
145,0.666667,3.0,5.2,2.3
146,0.555556,2.5,5.0,1.9
147,0.611111,3.0,5.2,2.0
148,0.527778,3.4,5.4,2.3


#### standardization
$$
z=x_{std}=\frac{x-\mu_x}{\sigma_x}
$$

In [177]:
#initialize the datasets

x=pd.DataFrame(data=iris["data"],columns=iris["feature_names"])

In [189]:
def standardization(y,data_name):
    return ((y-data_name["sepal length (cm)"]).mean()/data_name["sepal length (cm)"].std())

x["sepal length (cm)"].apply(lambda j:standardization(j,x))

0     -0.897674
1     -1.139200
2     -1.380727
3     -1.501490
4     -1.018437
         ...   
145    1.034539
146    0.551486
147    0.793012
148    0.430722
149    0.068433
Name: sepal length (cm), Length: 150, dtype: float64

or

In [185]:
x["sepal length (cm)"] = (x["sepal length (cm)"]-x["sepal length (cm)"].mean())/(x["sepal length (cm)"].std())
x

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,-0.897674,3.5,1.4,0.2
1,-1.139200,3.0,1.4,0.2
2,-1.380727,3.2,1.3,0.2
3,-1.501490,3.1,1.5,0.2
4,-1.018437,3.6,1.4,0.2
...,...,...,...,...
145,1.034539,3.0,5.2,2.3
146,0.551486,2.5,5.0,1.9
147,0.793012,3.0,5.2,2.0
148,0.430722,3.4,5.4,2.3
